In [1]:
import numpy as np 
import pandas as pd 
import pymysql
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn import preprocessing
%matplotlib notebook
import matplotlib.pyplot as plt, mpld3
from matplotlib import rcParams
from matplotlib.cm import rainbow
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import State, Input, Output
import dash_bootstrap_components as dbc
import dash_auth
from flask import Flask, redirect
from pathlib import Path
import dash_table
import plotly.graph_objs as go
engine = create_engine('mysql+pymysql://root:@localhost/student')
mycursor= engine.connect()
colors = ['lightslategray',] * 5
colors[1] = 'crimson'


def generate_graph(dataframe,i):
    
    return  html.Div(
                [
                    html.H3(i),
                    dcc.Graph(
                        figure={
                            "data": [
                               go.Bar(
                                    x=dataframe["GRADE"].unique(),
                                    y=dataframe["GRADE"].value_counts(),
                                    marker_color=colors
                                )]})
                ])
def generate_piechart(dataframe,f):
    w=len(dataframe)
    lis=[]
    lis.append(f)
    passs=w-f
    lis.append(passs)
    print(lis)
    return  html.Div(
                [
                    html.H3('passs% vs fail%'),
                    dcc.Graph(
                        figure={
                            "data": [
                               go.Pie(
                                    values=lis,
                                    labels=['fail%','pass%']
                                )]})
                ])
def generate_table(dataframe):
    max_rows=len(dataframe)
    return dbc.Table.from_dataframe(dataframe, striped=True, bordered=True, hover=True)
def importdata(): 
    balance_data = pd.read_excel('dataset.xlsx') 
    data_tests=pd.read_excel('testdata.xlsx')
    #original=data_tests
    
    #balance_data = balance_data.apply(le.fit_transform)
    #data_tests = data_tests.apply(le.fit_transform)
    return balance_data,data_tests

def splitdataset(balance_data): 

    # Separating the target variable 
    X = balance_data.values[:, 2:4] 
    Y = balance_data.values[:, 4]
    #print(X)
    # Splitting the dataset into train and test 
    X_train, X_test, y_train, y_test = train_test_split(  X, Y, test_size = 0.3, random_state = 100) 
    return X, Y, X_train, X_test, y_train, y_test 
def train_using_gini(X_train, X_test, y_train,y_test): 
    
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, min_samples_leaf=2) 
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 
# Function to perform training with entropy. 
def tarin_using_entropy(X_train, X_test, y_train): 

    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( criterion = "entropy", random_state = 100, max_depth = 3, min_samples_leaf = 2) 
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy 
def predictionres(data_tests, clf_object): 
    originaldata=data_tests
    # Predicton on test with giniIndex 
    datapred=[]
    #print(data_tests)
    #print(clf_object)
    datay_pred = clf_object.predict(data_tests) 
    #print("Predicted values:") 
    #print(datay_pred)
    datapred=list( datay_pred)
    #print(datapred)
    return datapred

#def main(): 
# Building Phase 
#le = preprocessing.LabelEncoder()
data,data_test = importdata() 
X, Y, X_train, X_test, y_train, y_test = splitdataset(data) 
clf_gini = train_using_gini(X_train, X_test, y_train,y_test) 
clf_entropy = tarin_using_entropy(X_train, X_test, y_train) 
app12= dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app12.config['suppress_callback_exceptions']=True
app12.layout =html.Div(children=[
        html.H1('STUDENT PERFORMANCE ANALYSIS SYSTEM USING MACHINE LEARNING',style={'text':'bold','textAlign': 'center','color':'red'}),
        html.Br(),html.Br(),
        html.Div([html.A('HOME PAGE', href='http://127.0.0.1:2020/', target='_self')]),html.Br(),
        html.Label(' '),
        html.Label('Select Year: '),
        dcc.Dropdown(
            id='d1',style={'vertical-align': 'middle','display': 'inline-block','textAlign': 'center','height': '30px', 'width': '200px'},
        options=[
            {'label': 'I', 'value': 'I'},
            {'label': 'II', 'value': 'II'},
            {'label': 'III', 'value': 'III'},
            {'label': 'IV', 'value': 'IV'}
        ]), html.Label(' '),html.Label(' '),html.Label(' '),
        html.Label('Select Semister: '),
        dcc.Dropdown(
        id='d2',style={'vertical-align': 'middle','display': 'inline-block','height': '30px', 'width': '200px'},
        options=[
            {'label': 'I', 'value': 'I'},
            {'label': 'II', 'value': 'II'},
        ]),html.Label(' '),html.Label(' '),html.Label(' '),
        html.Label('Select Branch : '),
       dcc.Dropdown(
        id='d3' ,style={'vertical-align': 'middle','display': 'inline-block','height': '30px', 'width': '200px'},
        options=[
            {'label': 'CSE', 'value': 'CSE'},
            {'label': 'ECE', 'value': 'ECE'},
            {'label': 'EEE', 'value': 'EEE'},
            {'label': 'MECH', 'value': 'MECH'},
            {'label' : 'CIVIL', 'value': 'CIVIL'}
        ]),html.Label(' '),html.Label(' '),html.Label(' '),
        html.Label('Select Subject: '),
        dcc.Dropdown(
        id='d4',style={'vertical-align': 'middle','display': 'inline-block','height': '30px', 'width': '200px'}
        ),html.Br(),html.Br(),
        html.Div(dbc.Button('predict', id='button3',color="primary", className="mr-1",n_clicks=0),style={'textAlign': 'center','color':'gray'}),
        html.Br(),html.Br(),html.Br(),html.Br(),
        html.Div(id='page1',style={'vertical-align': 'middle','display': 'inline-block','textAlign': 'center','backgroundColor':'white'})])
@app12.callback(
    dash.dependencies.Output('d4', 'options'),
    [dash.dependencies.Input('d1','value'),dash.dependencies.Input('d2','value'),dash.dependencies.Input('d3','value')],
    state=[State('d1','value'),State('d2','value'),State('d3','value')])
def update_options(d1,d1value,d2,d2value,d3,d3value):
    options=[]
    a=d1value
    b=d2value
    c=d3value
    #print(a,b,c)
    re=mycursor.execute("select subject from subjects where year='{b}' and sem='{a}' and branch='{c}'".format(b=b,a=a,c=c))
    for i in re:
        #print(i)
        for j in i:
            options.append(j)
    return [{'label': i, 'value': i} for i in options]
@app12.callback(
    dash.dependencies.Output('page1', 'children'),
    [dash.dependencies.Input('button3','n_clicks')],
    state=[State('d4','value')])
def compute20(n_clicks,d4value):
    #print("hello")
    if(int(n_clicks)>0 and d4value):
            #print("hi",d4value)
            pk=[]
            ds=mycursor.execute("select * from `{}`".format(d4value))
            df = pd.DataFrame(ds, columns=['Rollnumber','subject','mid1','mid2'])
            stulis10=df
            stulis1 = df.values[:, 2:4] 
            df3 = pd.DataFrame(stulis1)
            stuliresg=predictionres(df3,clf_gini)
            stulis10['GRADE']=np.array(stuliresg)
            f=stuliresg.count('F')
            return html.Br(),html.Label('using gini index'),html.Div([generate_table(stulis10,)]),html.Br(),html.Div([generate_graph(stulis10,'count of each grade')]),html.Br(),html.Br(),html.Br(),html.Br(),html.Div([generate_piechart(stulis10,f)])
            
                
                

In [ ]:
if __name__ == '__main__':
    app12.run_server(port=2017)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:2017/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /assets/stylesheet.css?m=1584007644.0875306 HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_2m1583924370.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_2m1583924370.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_2m1583924370.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /_dash-component-suites/dash_table/bundle.v4_6_1m1583924377.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v0_8_3m1583924366.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:15] "GET /_dash-component-suites/dash_render

[0, 3]


127.0.0.1 - - [03/Apr/2020 00:59:29] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_8_1m1582838719.js HTTP/1.1" 200 -
